ПАКЕТ SURPRISE

используйте данные MovieLens 1M
можно использовать любые модели из пакета
получите RMSE на тестовом сете 0.87 и ниже

Комментарий преподавателя :
В ДЗ на датасет 1М можешь не хватить RAM. Можно сделать на 100K. Качество RMSE предлагаю считать на основе CrossValidation (5 фолдов), а не отложенном датасете

In [1]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise.model_selection import train_test_split
from surprise import accuracy

In [2]:
#Мой ноутбук Core i5-8265U 1.6 RAN 12Gb, Win10 Pro, прожувать 1М датасет не смог, по этому я использовал small датасет
df_ratings = pd.read_csv('ratings.csv')
df_ratings.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


In [3]:
df_for_surpise = df_ratings[['userId', 'movieId', 'rating']]
df_for_surpise.columns = ['uid', 'iid', 'rating']
df_for_surpise.head()

,uid,iid,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [4]:
df_ratings.rating

0         4.0
1         4.0
2         4.0
3         5.0
4         5.0
         ... 
100831    4.0
100832    5.0
100833    5.0
100834    5.0
100835    3.0
Name: rating, Length: 100836, dtype: float64

In [5]:
max_df_rating = df_ratings.rating.max()
min_df_rating = df_ratings.rating.min()

In [6]:
reader = Reader(rating_scale=(min_df_rating, max_df_rating))
dataset = Dataset.load_from_df(df_for_surpise, reader)

In [7]:
trainset, testset = train_test_split(dataset, test_size=0.2)

### KNNBasic

In [8]:
algo = KNNBasic(k=40, sim_options={'name': 'cosine', 'user_based': True})
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9788


0.9788235181097763

Делать из 0.97 мне показалось нецелесообразно. Пробую следующий алгоритм.

### KNNWithMeans

In [9]:
algo = KNNWithMeans(k=40, sim_options={'name': 'cosine', 'user_based': True})
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9028


0.9028398898360052

Уже лучьше, но может следующий алгоритм на дефолте покажет лучший результат?

### KNNWithZScore

In [10]:
algo = KNNWithZScore(k=40, sim_options={'name': 'cosine', 'user_based': True})
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.8993


0.899266082369972

Результат немного лучше, смотрю далее последний.

### KNNBaseline

Это алгоритм по дефолту показал 0.88. Под него я и стал подбирать параметры.

In [11]:
#словари параметров для перебора
similarity_measures = ['cosine', 'msd', 'pearson', 'pearson_baseline']
bsl__measures = ['als', 'sgd']

#можно было бы поиграться параметрами 'n_epochs','reg_u' и 'reg_i', но в слепую подбирать я не стал
bsl_options = {'method': 'als'}

sim_options={'name': 'cosine',
             'user_based': True}

for met_bsl in bsl__measures:
    print ('Baselines estimatese:', '\033[1m' + met_bsl + '\033[0m', '\n')
    
    for name_sim in similarity_measures:
        sim_options['name'] = name_sim
        algo = KNNBaseline(k=40, sim_options = sim_options, bsl_options = bsl_options)
        algo.fit(trainset)
        predictions = algo.test(testset)
        acc = accuracy.rmse(predictions)
        print ('The name of the similarity to use:', '\033[1m' + name_sim + '\033[0m', '\n')
    bsl_options['method'] = 'sgd'

#Заморачитваться со словарем параметров, что бы выбрать из него кодом лучший результат я поленился :)

Baselines estimatese: als 

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.8803
The name of the similarity to use: cosine 

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8754
The name of the similarity to use: msd 

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.8814
The name of the similarity to use: pearson 

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8804
The name of the similarity to use: pearson_baseline 

Baselines estimatese: sgd 

Estimating biases using sgd...
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.8779
The name of the similarity to use: cosine 

Estimating biases using sgd...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE

## Лучший результат:
### Baselines estimatese: sgd
### The name of the similarity to use: msd
### RMSE: 0.8732